## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：

![问题1](./fig/hw3/问题1.png)

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

In [ ]:
from graphviz import Digraph

class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None
    
    def insert(self, val):
        self.root = self._insert(self.root, val)
    
    def _insert(self, root, val):
        if not root:
            return TreeNode(val)
        if val < root.val:
            root.left = self._insert(root.left, val)
        else:
            root.right = self._insert(root.right, val)
        return root
    
    def delete(self, val):
        self.root = self._delete(self.root, val)
    
    def _delete(self, root, val):
        if not root:
            return root
        if val < root.val:
            root.left = self._delete(root.left, val)
        elif val > root.val:
            root.right = self._delete(root.right, val)
        else:
            # 节点删除逻辑
            if not root.left:
                return root.right
            elif not root.right:
                return root.left
            else:
                # 找到右子树的最小节点替换当前节点
                temp = self._find_min(root.right)
                root.val = temp.val
                root.right = self._delete(root.right, temp.val)
        return root
    
    def _find_min(self, root):
        current = root
        while current.left:
            current = current.left
        return current
    
    def visualize(self, filename):
        dot = Digraph()
        if self.root:
            self._add_nodes_edges(self.root, dot)
        dot.render(f'fig/{filename}', format='png', cleanup=True)
        print(f"树结构已保存为 {filename}.png")

    def _add_nodes_edges(self, node, dot):
        dot.node(str(node.val))
        if node.left:
            dot.edge(str(node.val), str(node.left.val))
            self._add_nodes_edges(node.left, dot)
        if node.right:
            dot.edge(str(node.val), str(node.right.val))
            self._add_nodes_edges(node.right, dot)

# 构建初始二叉搜索树
bst = BST()
data = [48, 33, 49, 47, 42, 46, 32]
for num in data:
    bst.insert(num)
bst.visualize("hw3_original_tree")

# 删除节点 33
bst.delete(33)
bst.visualize("hw3_tree_after_deletion")

树结构已保存为 hw3_original_tree.png
树结构已保存为 hw3_tree_after_deletion.png


you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：

![问题2](./fig/hw3/问题2.png)

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：参考大模型

### 算法分析
使用贪婪算法解决这个问题：
1. 将所有课程按照结束时间排序
2. 依次为每个教室安排尽可能多的课程
3. 对于每个教室，选择当前结束时间最早且不与已安排课程冲突的课程

code：

In [12]:
from collections import defaultdict
import heapq

def parse_courses(courses_str):
    # 移除所有空格和换行符
    courses_str = courses_str.replace(' ', '').replace('\n', '')
    # 移除最外层的方括号
    courses_str = courses_str.strip('[]')
    # 分割每个课程时间对
    courses_items = courses_str.split('),(')
    # 处理第一个和最后一个元素
    courses_items[0] = courses_items[0].lstrip('(')
    courses_items[-1] = courses_items[-1].rstrip(')')
    
    courses = []
    for item in courses_items:
        start_str, end_str = item.split(',')
        courses.append((start_str, end_str))
    return courses

def schedule_courses(courses):
    # 将时间转换为分钟数，便于比较
    courses_minutes = []
    for i, (start, end) in enumerate(courses):
        start_parts = start.split(':')
        end_parts = end.split(':')
        start_minutes = int(start_parts[0]) * 60 + int(start_parts[1])
        end_minutes = int(end_parts[0]) * 60 + int(end_parts[1])
        courses_minutes.append((start_minutes, end_minutes, i))
    
    # 按开始时间排序
    courses_minutes.sort()
    
    # 使用优先队列跟踪当前正在进行的课程
    rooms = []
    room_assignments = defaultdict(list)
    room_count = 0
    
    for start, end, course_idx in courses_minutes:
        # 检查是否有可用的教室
        if rooms and rooms[0][0] <= start:
            # 使用最早结束的教室
            _, room_idx = heapq.heappop(rooms)
            heapq.heappush(rooms, (end, room_idx))
            room_assignments[room_idx].append((course_idx, start, end))
        else:
            # 需要新教室
            room_count += 1
            heapq.heappush(rooms, (end, room_count))
            room_assignments[room_count].append((course_idx, start, end))
    
    print(f"最少需要 {room_count} 间教室")
    for room in range(1, room_count + 1):
        print(f"\n教室 {room} 的安排:")
        for course_idx, start, end in sorted(room_assignments[room], key=lambda x: x[1]):
            original_interval = courses[course_idx]
            print(f"课程 {course_idx + 1}: {original_interval[0]} - {original_interval[1]}")
    
    return room_assignments

if __name__ == "__main__":
    courses_str = "[(9:00,12:30), (11:00,14:00), (13:00,14:30), (9:00,10:30), (13:00,14:30), (14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]"
    courses = parse_courses(courses_str)
    classrooms = schedule_courses(courses)

最少需要 3 间教室

教室 1 的安排:
课程 4: 9:00 - 10:30
课程 2: 11:00 - 14:00
课程 6: 14:00 - 16:30

教室 2 的安排:
课程 9: 9:00 - 10:30
课程 3: 13:00 - 14:30
课程 7: 15:00 - 16:30

教室 3 的安排:
课程 1: 9:00 - 12:30
课程 5: 13:00 - 14:30
课程 8: 15:00 - 16:30


## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法分析
这个问题可以通过动态规划来解决。
定义一个数组`dp`，其中`dp[i]`表示爬到第i阶的方法总数。
- 如果只有一阶楼梯，只有一种方法，即爬一阶：`dp[1]=1`
- 如果有两阶楼梯，有两种方法：爬一阶两次或者直接爬两阶：`dp[2]=2`
对于i≥3爬到i阶的方法可以通过一下两种方式得到：
1. 从第i-1阶爬一阶到达第i阶。
2. 从第i-2阶爬两阶到达第i阶。
因此，递推公式为：   `dp[i]=dp[i−1]+dp[i−2]`

In [13]:
# add your code here
# your algorithm time complexity is: O(n)
def climb_stairs(n):
    if n <= 2:
        return n
    
    dp = [0] * (n + 1)
    dp[1], dp[2] = 1, 2
    
    for i in range(3, n + 1):
        dp[i] = dp[i - 1] + dp[i - 2]
    
    return dp[n]

# 示例
n = 5
print(f"爬上 {n} 阶楼梯的方法总数是: {climb_stairs(n)}")

爬上 5 阶楼梯的方法总数是: 8


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法设计思路
可以使用动态规划算法来解决这个问题：
1. 创建一个二维数组`dp[i][j]`，其中`i`表示考虑前`i`个物品，`j`表示背包容量为`j`时的最大价值
2. 对于每个物品，我们有两种选择：放入或不放入
3. 状态转移方程为：`dp[i][j] = max(dp[i-1][j], dp[i-1][j-weight[i]] + value[i])`
4. 时间复杂度为O(nW)，其中n是物品数量，W是背包容量

In [1]:
# add your code here
# your algorithm time complexity is:O(nW)
def knapsack_0_1(items, capacity):
    n = len(items)
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        weight, value = items[i-1]
        for j in range(capacity + 1):
            if weight <= j:
                dp[i][j] = max(dp[i-1][j], dp[i-1][j-weight] + value)
            else:
                dp[i][j] = dp[i-1][j]
    
    # 找出选择了哪些物品
    selected_items = []
    j = capacity
    for i in range(n, 0, -1):
        if dp[i][j] != dp[i-1][j]:
            selected_items.append(i-1)
            j -= items[i-1][0]
    
    return dp[n][capacity], selected_items

# 测试
items = [(5, 10), (4, 40), (6, 30), (3, 50)]
capacity = 9
max_value, selected = knapsack_0_1(items, capacity)

print(f"最大价值: {max_value}")
print(f"选择的物品索引: {selected}")
print("选择的物品详情:")
for idx in selected:
    print(f"物品 {idx+1}: 重量={items[idx][0]}, 价值={items[idx][1]}")

最大价值: 90
选择的物品索引: [3, 1]
选择的物品详情:
物品 4: 重量=3, 价值=50
物品 2: 重量=4, 价值=40


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法设计思路
1. 对于每个位置i，能接的雨水量取决于这个位置左右两边最高柱子的较小值与当前位置高度的差值
2. 可以使用双指针的方法来解决这个问题
3. 维护两个指针`left`和`right`，以及两个变量`leftMax`和`rightMax`记录左右两边的最大高度
4. 从两端向中间移动，根据`leftMax`和`rightMax`的大小关系决定从哪边移动

### 时间复杂度
O(n),只需要遍历一次数组

In [3]:
# add your code here
# your algorithm time complexity is:O(n)

def rainfall_amount(height):
    if not height:
        return 0
    
    left, right = 0, len(height) - 1
    left_max, right_max = 0, 0
    water = 0

    while left < right:
        left_max = max(left_max, height[left])
        right_max = max(right_max, height[right])

        if left_max < right_max:
            water += left_max - height[left]
            left += 1
        else:
            water += right_max - height[right]
            right -= 1

    return water

# height = [0,1,0,2,1,0,1,3,2,1,2,1]
height = [4,2,0,3,2,5]
print(f"能接的雨水量: {rainfall_amount(height)}")

能接的雨水量: 9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

### 算法设计思路
1. 计算每只股票的单位投资金额的收益率
2. 按照单位收益率从高到低对股票进行排序
3. 从收益率最高的股票开始投资，直到达到预算限制

### 时间复杂度
- 排序操作：O(nlogn)
- 分配投资：O(n)
- 总时间复杂度：O(nlogn)

In [8]:
# add your code here
# your algorithm time complexity is:O(nlogn)

class Stock:
    def __init__(self, name, amount, rate):
        self.name = name
        self.amount = amount
        self.rate = rate

def max_profit(stocks, budget):
    if budget <= 0 or not stocks:
        return {}, 0
    
    # 直接按照收益率排序
    sorted_stocks = sorted(stocks, key=lambda x: x.rate, reverse=True)

    total_profit = 0
    investments = {}
    remaining_budget = budget

    for stock in sorted_stocks:
        if remaining_budget <= 0:
            break

        invest_amount = min(stock.amount, remaining_budget)
        return_amount = invest_amount * (stock.rate/100)

        investments[stock.name] = invest_amount
        total_profit += return_amount
        remaining_budget -= invest_amount

    return investments, total_profit

# 测试代码
stocks = [
    Stock("A", 5000, 10),
    Stock("B", 3000, 8),
    Stock("C", 2000, 12)
]

budget = 8000

investments, total_profit = max_profit(stocks, budget)

print("最优投资方案:")
for stock_name, amount in investments.items():
    stock = next(s for s in stocks if s.name == stock_name)
    print(f"股票{stock_name}: {amount:.2f} 美元 (收益率: {stock.rate}%)")

print(f"最大预期收益: {total_profit:.2f} 美元")

最优投资方案:
股票C: 2000.00 美元 (收益率: 12%)
股票A: 5000.00 美元 (收益率: 10%)
股票B: 1000.00 美元 (收益率: 8%)
最大预期收益: 820.00 美元


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：


### 算法设计思路
二叉搜索树的中序遍历应该是一个递增序列。当有两个节点被错误交换后，中序遍历序列中会出现1-2个位置的值不符合升序规则。
算法步骤；
1. 通过中序遍历找到不满足升序条件的两个节点。
2. 交换这两个节点的值来恢复二叉搜索树
3. 可能有两种情况：
    - 被交换的节点相邻：序列中只有一个降序点
    - 被交换的节点不相邻：序列中有两个降序点

### 时间复杂度
O(n)，需要遍历所有节点  

In [5]:
# 时间复杂度：O(n)，只需一次中序遍历

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class BST:
    def recoverTree(self, root):
        self.first = None
        self.second = None 
        self.prev = TreeNode(float('-inf')) 
        
        def inorder(node):
            if not node:
                return
            
            inorder(node.left)
            
            # 检查当前节点是否违反二叉搜索树特性
            if self.prev.val >= node.val:
                if not self.first:
                    self.first = self.prev

                self.second = node
            
            self.prev = node
            
            inorder(node.right)
        
        inorder(root)

        self.first.val, self.second.val = self.second.val, self.first.val

def print_inorder(root):
    result = []
    
    def inorder(node):
        if node:
            inorder(node.left)
            result.append(node.val)
            inorder(node.right)
    
    inorder(root)
    return result

# [1,3,null,null,2] -> [3,1,null,null,2]
root1 = TreeNode(1)
root1.left = TreeNode(3)
root1.left.right = TreeNode(2)

print("输入：", print_inorder(root1))
BST().recoverTree(root1)
print("输出：", print_inorder(root1))

# [3,1,4,null,null,2] -> [2,1,4,null,null,3]
# root2 = TreeNode(3)
# root2.left = TreeNode(1)
# root2.right = TreeNode(4)
# root2.right.left = TreeNode(2)

# print("输入：", print_inorder(root2))
# BST().recoverTree(root2)
# print("输出：", print_inorder(root2))

输入： [3, 2, 1]
输出： [1, 2, 3]


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

### 算法设计思路
该题可以使用深度优先搜索来解决：
1. 使用递归方式遍历二叉树
2. 在遍历过程中，维护一个变量`current_num`代表从根节点到当前节点形成的数字
3. 当遇到叶节点时，将当前路径形成的数字加入总和
4. 对于非叶节点，递归处理其左右子树，并将结果相加

### 时间复杂度
对于树中的每个节点，仅需访问一次，因此时间复杂度为O(n)

In [7]:
# add your code here
# your algorithm time complexity is:O(n)

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def sum(root):
    def dfs(node, current_num):
        if not node:
            return 0
        
        current_num = current_num * 10 + node.val
        
        if not node.left and not node.right:
            return current_num
        
        return dfs(node.left, current_num) + dfs(node.right, current_num)
    
    return dfs(root, 0)
    

# 测试[1,2,3]
root1 = TreeNode(1)
root1.left = TreeNode(2)
root1.right = TreeNode(3)
print(f"示例1结果: {sum(root1)}")

# 测试[4,9,0,5,1]
root2 = TreeNode(4)
root2.left = TreeNode(9)
root2.right = TreeNode(0)
root2.left.left = TreeNode(5)
root2.left.right = TreeNode(1)
print(f"示例2结果: {sum(root2)}")

示例1结果: 25
示例2结果: 1026


## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea

### 递归
二叉树的对称性可以归结为检查根节点的左子树和右子树是否互为镜像  
- 设计思路：
    - 两棵树互为镜像意味着它们的根节点值相同
    - 左树的左子树与右树的右子树互为镜像
    - 左树的右子树与左子树互为镜像
- 递归终止条件：
    - 如果两个节点都为空，返回true
    - 如果只有一个节点为空，返回false
    - 如果两个节点值不同，返回false
- **时间复杂度**：O(n)，每个节点只访问一次

### 迭代
使用队列实现广度优先搜索，同时比较对称位置的节点
- 设计思路：
    - 初始时将根节点的左右子树入队
    - 每次从队列取出两个节点进行比较‘
    - 如果它们相同，则将左节点的左子节点与右节点的右子节点入队
    - 同时将左节点的右子节点与右节点的左子节点入队
- 判断条件：
    - 两节点都为空：继续检查队列中其他节点
    - 有一个节点为空或值不同：树不对称，返回false
    - 队列为空且没有返回false:树对称，返回true
- **时间复杂度**:O(n),需要访问所有节点

In [6]:
# add your code here
# your algorithm time complexity is:O(n)

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# 递归
def is_symmetric_recursive(root):
    if not root:
        return True
    
    def is_mirror(left, right):
        if not left and not right:
            return True

        if not left or not right:
            return False

        if left.val != right.val:
            return False

        return is_mirror(left.left, right.right) and is_mirror(left.right, right.left)
    
    return is_mirror(root.left, root.right)

# 迭代
def is_symmetric_iterative(root):

    if not root:
        return True
    
    queue = [(root.left, root.right)]
    
    while queue:
        left, right = queue.pop(0)
        
        if not left and not right:
            continue
        
        if not left or not right or left.val != right.val:
            return False
        
        queue.append((left.left, right.right))
        queue.append((left.right, right.left))
    
    return True

def build_tree_example1():
    # [1,2,2,3,4,4,3]
    root = TreeNode(1)
    root.left = TreeNode(2)
    root.right = TreeNode(2)
    root.left.left = TreeNode(3)
    root.left.right = TreeNode(4)
    root.right.left = TreeNode(4)
    root.right.right = TreeNode(3)
    return root

def build_tree_example2():
    # [1,2,2,null,3,null,3]
    root = TreeNode(1)
    root.left = TreeNode(2)
    root.right = TreeNode(2)
    root.left.right = TreeNode(3)
    root.right.right = TreeNode(3)
    return root

if __name__ == "__main__":
    tree1 = build_tree_example1()
    print("1 (递归):", is_symmetric_recursive(tree1))
    print("1 (迭代):", is_symmetric_iterative(tree1))
    
    tree2 = build_tree_example2()
    print("2 (递归):", is_symmetric_recursive(tree2)) 
    print("2 (迭代):", is_symmetric_iterative(tree2)) 

1 (递归方法): True
1 (迭代方法): True
2 (递归方法): False
2 (迭代方法): False


idea：

## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3  
idea:

### 算法设计思路
这是一个经典的图搜索问题，可以使用深度优先搜索来解决
1. 遍历整个网格，当找到一个值为1的单元格时，从该单元格开始进行深度优先搜索
2. 在深度优先搜索过程中，将所有与当前陆地相连的陆地标记为已访问
3. 每完成一次完整的深度优先搜索，岛屿计数器加1
4. 继续遍历网格中剩余未访问的单元格，重复上述过程。

**以下参考大模型：**
但是，对于大型网格，递归调用可能会超过python的默认递归深度限制，  
所以可以使用迭代方法替代递归，使用栈来模拟DFS过程，迭代方法更可靠

### 时间复杂度
- 假设网格大小为m*n
- 每个单元格最多被访问一次
- 总时间复杂度：O(m×n)

In [5]:
# add your code here
# your algorithm time complexity is:O(m×n)

def count_islands(grid):
    if not grid or not grid[0]:
        return 0
    
    rows,cols=len(grid),len(grid[0])
    count=0

    def dfs(i,j):
        stack=[(i,j)]
        grid[i][j]="0"
        
        while stack:
            curr_i,curr_j=stack.pop()

            directions=[(1,0),(-1,0),(0,1),(0,-1)]

            for di,dj in directions:
                ni,nj=curr_i+di,curr_j+dj

                if 0<=ni<rows and 0<=nj<cols and grid[ni][nj]=="1":
                    grid[ni][nj]="0"
                    stack.append((ni,nj))

    for i in range(rows):
        for j in range(cols):
            if grid[i][j]=="1":
                count+=1
                dfs(i,j)

    return count

# 测试例子1
grid1 = [
    ["1","1","1","1","0"],
    ["1","1","0","1","0"],
    ["1","1","0","0","0"],
    ["0","0","0","0","0"]
]
print(f"岛屿数量: {count_islands(grid1)}")

# 测试例子2
grid2 = [
    ["1","1","0","0","0"],
    ["1","1","0","0","0"],
    ["0","0","1","0","0"],
    ["0","0","0","1","1"]
]
print(f"岛屿数量: {count_islands(grid2)}")

岛屿数量: 1
岛屿数量: 3
